In [1]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

Uso de las tablas de clean para feature engineering

In [2]:
#Cargar los datos
dim_user_plans=pd.read_csv("../data/clean/dim_user_plans.csv")
fct_sales=pd.read_csv("../data/clean/fct_sales.csv")



In [3]:
#Añadir la columna month y year
fct_sales['date']=pd.to_datetime(fct_sales['date'])
fct_sales['month']=fct_sales['date'].dt.month
fct_sales['year']=fct_sales['date'].dt.year
#Quitamos la columna date
fct_sales=fct_sales.drop(columns=['date'])
#Agrupar por mes,año y user_id
fct_sales=fct_sales.groupby(['month','year','user_id']).sum().reset_index()
fct_sales.head()


,month,year,user_id,total_minutes,total_messages,total_mb
0,1,2018,1000,0.0,0.0,0.0
1,1,2018,1001,0.0,0.0,0.0
2,1,2018,1002,0.0,0.0,0.0
3,1,2018,1003,0.0,0.0,0.0
4,1,2018,1004,0.0,0.0,0.0


In [4]:
print(fct_sales.loc[fct_sales['month']==12])


      month  year  user_id  total_minutes  total_messages  total_mb
5390     12  2018     1000         116.83            11.0   1901.47
5391     12  2018     1001         392.93            44.0  19369.18
5392     12  2018     1002         363.24            41.0  14396.24
5393     12  2018     1003        1041.00            50.0  27044.14
5394     12  2018     1004         403.53            31.0  21389.29
...     ...   ...      ...            ...             ...       ...
5875     12  2018     1495         467.47             0.0  24097.40
5876     12  2018     1496         278.61            11.0  16438.99
5877     12  2018     1497         276.53            50.0  11106.55
5878     12  2018     1498         324.77             0.0  23137.69
5879     12  2018     1499         468.10             0.0  22059.21

[490 rows x 6 columns]


In [5]:
#Tabla que 
#Añadir el costo total viendo si un usuario uso más de lo que abastece su plan por cada mes
fct_sales=pd.merge(fct_sales,dim_user_plans,on='user_id',how='left')
fct_sales['exceso_messages'] = fct_sales.apply(lambda row: max(0, row['total_messages'] - row['messages_included']), axis=1)
fct_sales['exceso_mb_used'] = fct_sales.apply(lambda row: max(0, row['total_mb'] - row['mb_per_month_included']), axis=1)
fct_sales['exceso_minutes'] = fct_sales.apply(lambda row: max(0, row['total_minutes'] - row['minutes_included']), axis=1)
fct_sales['costo_extra_messages'] = fct_sales['exceso_messages'] * fct_sales['usd_per_message']
fct_sales['costo_extra_mb_used'] = fct_sales['exceso_mb_used'] * fct_sales['usd_per_mb']
fct_sales['costo_extra_minutes'] = fct_sales['exceso_minutes'] * fct_sales['usd_per_minute']
fct_sales['costo_extra'] = fct_sales['costo_extra_messages'] + fct_sales['costo_extra_mb_used'] + fct_sales['costo_extra_minutes']
fct_sales['total_cost'] = fct_sales['usd_monthly_pay'] + fct_sales['costo_extra']
print(fct_sales.head())


   month  year  user_id  total_minutes  total_messages  total_mb first_name  \
0      1  2018     1000            0.0             0.0       0.0   Anamaria   
1      1  2018     1001            0.0             0.0       0.0     Mickey   
2      1  2018     1002            0.0             0.0       0.0     Carlee   
3      1  2018     1003            0.0             0.0       0.0   Reynaldo   
4      1  2018     1004            0.0             0.0       0.0    Leonila   

   last_name  age                                   city  ... usd_per_minute  \
0      Bauer   45  Atlanta-Sandy Springs-Roswell, GA MSA  ...           0.01   
1  Wilkerson   28        Seattle-Tacoma-Bellevue, WA MSA  ...           0.03   
2    Hoffman   36   Las Vegas-Henderson-Paradise, NV MSA  ...           0.03   
3    Jenkins   52                          Tulsa, OK MSA  ...           0.03   
4   Thompson   40        Seattle-Tacoma-Bellevue, WA MSA  ...           0.03   

  plan_name  exceso_messages  exceso_mb_used

In [6]:
#Se añaden una feature que trabaje con city y tome la primera palabra (antes del espacio xq hay 2 palabras) después de la coma para tener el código de la ciudad
fct_sales['city_code']=fct_sales['city'].str.split(',').str[1].str.split(' ').str[1]

#Usar ordinal encoder para brindar un orden a las ciudades
fct_sales['city_code']=OrdinalEncoder().fit_transform(fct_sales[['city_code']]).astype(int)
fct_sales.head()

,month,year,user_id,total_minutes,total_messages,total_mb,first_name,last_name,age,city,...,plan_name,exceso_messages,exceso_mb_used,exceso_minutes,costo_extra_messages,costo_extra_mb_used,costo_extra_minutes,costo_extra,total_cost,city_code
0,1,2018,1000,0.0,0.0,0.0,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",...,ultimate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,7
1,1,2018,1001,0.0,0.0,0.0,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",...,surf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,40
2,1,2018,1002,0.0,0.0,0.0,Carlee,Hoffman,36,"Las Vegas-Henderson-Paradise, NV MSA",...,surf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,23
3,1,2018,1003,0.0,0.0,0.0,Reynaldo,Jenkins,52,"Tulsa, OK MSA",...,surf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,28
4,1,2018,1004,0.0,0.0,0.0,Leonila,Thompson,40,"Seattle-Tacoma-Bellevue, WA MSA",...,surf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,40


In [7]:
fct_sales['reg_date'] = pd.to_datetime(fct_sales['reg_date'])

# Extrae mes y año de 'reg_date'
fct_sales['reg_month'] = fct_sales['reg_date'].dt.month
fct_sales['reg_year'] = fct_sales['reg_date'].dt.year

fct_sales = fct_sales[(fct_sales['reg_year'] <= fct_sales['year'])  & (fct_sales['reg_month'] <= fct_sales['month'])]



# Muestra el resultado
print(fct_sales.head())


    month  year  user_id  total_minutes  total_messages  total_mb  first_name  \
3       1  2018     1003           0.00             0.0      0.00    Reynaldo   
11      1  2018     1011           0.00             0.0      0.00      Halina   
19      1  2018     1019           0.00             0.0      0.00      Shizue   
28      1  2018     1029           0.00             0.0      0.00    Franklyn   
41      1  2018     1042         106.83             0.0   1854.93  Clementina   

     last_name  age                                             city  ...  \
3      Jenkins   52                                    Tulsa, OK MSA  ...   
11       Henry   73                         Cleveland-Elyria, OH MSA  ...   
19      Landry   34                             Jacksonville, FL MSA  ...   
28      Henson   59          Tampa-St. Petersburg-Clearwater, FL MSA  ...   
41  Mclaughlin   40  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD MSA  ...   

   exceso_mb_used exceso_minutes  costo_extra_mess

In [8]:
#Extraer mes y año de churn_date
fct_sales['churn_date'] = pd.to_datetime(fct_sales['churn_date'])
fct_sales['churn_month'] = fct_sales['churn_date'].dt.month
fct_sales['churn_year'] = fct_sales['churn_date'].dt.year
fct_sales=fct_sales[(fct_sales['churn_year'] >= fct_sales['year'])  & (fct_sales['churn_month'] >= fct_sales['month'])]
print(fct_sales.head())

    month  year  user_id  total_minutes  total_messages  total_mb  first_name  \
3       1  2018     1003           0.00             0.0      0.00    Reynaldo   
11      1  2018     1011           0.00             0.0      0.00      Halina   
19      1  2018     1019           0.00             0.0      0.00      Shizue   
28      1  2018     1029           0.00             0.0      0.00    Franklyn   
41      1  2018     1042         106.83             0.0   1854.93  Clementina   

     last_name  age                                             city  ...  \
3      Jenkins   52                                    Tulsa, OK MSA  ...   
11       Henry   73                         Cleveland-Elyria, OH MSA  ...   
19      Landry   34                             Jacksonville, FL MSA  ...   
28      Henson   59          Tampa-St. Petersburg-Clearwater, FL MSA  ...   
41  Mclaughlin   40  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD MSA  ...   

   costo_extra_messages costo_extra_mb_used  costo

In [9]:
#Cambiar la columna month y year a otras similares pero con ordinal Encoder
fct_sales['month']=OrdinalEncoder().fit_transform(fct_sales[['month']]).astype(int)
fct_sales['year']=OrdinalEncoder().fit_transform(fct_sales[['year']]).astype(int)

In [10]:
#Cambiar la columna plan_name a otra similar pero con ordinal Encoder
fct_sales['plan_name']=OrdinalEncoder().fit_transform(fct_sales[['plan_name']]).astype(int)

In [11]:
#cargar el one big table para ML a ml
one_big_table=fct_sales.copy()
one_big_table.to_csv('../data/ml/one_big_table.csv',index=False)